In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import gc
from typing import List
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader, random_split
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR

import riiideducation

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/riiid-train-data-multiple-formats/riiid_train.h5
/kaggle/input/riiid-train-data-multiple-formats/riiid_train.parquet
/kaggle/input/riiid-train-data-multiple-formats/riiid_train.jay
/kaggle/input/riiid-train-data-multiple-formats/riiid_train.feather
/kaggle/input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip
/kaggle/input/riiid-test-answer-prediction/lectures.csv
/kaggle/input/riiid-test-answer-prediction/example_test.csv
/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv
/kaggle/input/riiid-test-answer-prediction/train.csv
/kaggle/input/riiid-test-answer-prediction/questions.csv
/kaggle/input/riiid-test-answer-prediction/riiideducation/__init__.py
/kaggle/input/riiid-test-answer-prediction/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so


In [2]:
%%time

userActions = pd.read_pickle("../input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip")
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')

print("Train size:", userActions.shape)

Train size: (101230332, 10)
CPU times: user 5.9 s, sys: 9.79 s, total: 15.7 s
Wall time: 16.1 s


In [3]:
# only care about the question interactions
userActions = userActions.loc[userActions['content_type_id'] == 0]

gc.collect()

26

In [4]:
# does no care about correct answer
questions = questions.drop(columns=['correct_answer'])
questions.head()

,question_id,bundle_id,part,tags
0,0,0,1,51 131 162 38
1,1,1,1,131 36 81
2,2,2,1,131 101 162 92
3,3,3,1,131 149 162 29
4,4,4,1,131 5 162 38


In [5]:
gc.collect()

20

# Preprocess Question Tags

In [6]:
# convert tags to array of numbers
maximumTag = -math.inf
minimumTag = math.inf
def tagsToArray(x: str) -> List[int]:
    if x is np.nan:
        return []
    res = [int(tag) for tag in x.split()]
    global maximumTag
    global minimumTag
    maximumTag = max(maximumTag, *res)
    minimumTag = min(minimumTag, *res)
    return res
    

questions.tags = questions.tags.apply(tagsToArray)

questions.head()

,question_id,bundle_id,part,tags
0,0,0,1,"[51, 131, 162, 38]"
1,1,1,1,"[131, 36, 81]"
2,2,2,1,"[131, 101, 162, 92]"
3,3,3,1,"[131, 149, 162, 29]"
4,4,4,1,"[131, 5, 162, 38]"


In [7]:
print('min tag: ' + str(minimumTag))
print('max tag: ' + str(maximumTag))

min tag: 0
max tag: 187


In [8]:
def convertTagsToEncodedArray(x: List[int]) -> List[int]:
    global maximumTag
    encoded = np.zeros(maximumTag + 1)
    for tag in x:
        encoded[tag] = 1
    return encoded


questions.tags = questions.tags.apply(convertTagsToEncodedArray)

questions.head()

,question_id,bundle_id,part,tags
0,0,0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,2,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,3,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,4,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [9]:

interactionFeatures = ['timestamp', 'content_type_id', 'task_container_id', 'prior_question_elapsed_time', 'prior_question_had_explanation']
questionFeatures = ['bundle_id', 'part']

numberOfTrainingUsers = 1000

def toInt(x):
    if pd.isna(x) or x is None or x == np.nan:
        return -1
    return int(x)

def cleanUpUserInteractions(userInteractions, garbageCollect=False, removeNoneQuestions=False):
    # only look at interactions related to questions
    if garbageCollect:
        gc.collect()
    interactions = userInteractions
    
    if garbageCollect:
        gc.collect()
    
    if removeNoneQuestions:
        interactions = userInteractions.loc[userInteractions['content_type_id'] == 0]
    
    if garbageCollect:
        gc.collect()

    
    interactions['content_type_id'] = interactions['content_type_id'].apply(toInt)
    
    gc.collect()
    
    interactions['prior_question_elapsed_time'] = interactions['prior_question_elapsed_time'].apply(toInt)
    
    if garbageCollect:
        gc.collect()
    
    interactions['timestamp'] = interactions['timestamp'].apply(toInt)
    
    if garbageCollect:
        gc.collect()
    
    interactions['content_type_id'] = interactions['content_type_id'].apply(toInt)
    
    if garbageCollect:
        gc.collect()
    
    interactions['prior_question_had_explanation'] = interactions['prior_question_had_explanation'].apply(toInt)

    if garbageCollect:
        gc.collect()
    
    interactions[interactionFeatures] = interactions[interactionFeatures].astype('float').fillna(value = -1)
    
    if garbageCollect:
        gc.collect()

    
    return interactions
    

class UserDataset(Dataset):
    """Dataset class for column dataset.
    Args:
       cats (list of str): List of the name of columns contain
                           categorical variables.
       conts (list of str): List of the name of columns which 
                           contain continuous variables.
       y (Tensor, optional): Target variables.
       is_reg (bool): If the task is regression, set ``True``, 
                      otherwise (classification) ``False``.
       is_multi (bool): If the task is multi-label classification, 
                        set ``True``.
    """
    def __init__(self, userInteractions, questions):
        gc.collect()
        self.questions = {}
        self.tags = {}
        for index, row in questions.iterrows():
            question_id = row.question_id
            self.questions[question_id] = torch.from_numpy(row[questionFeatures].astype('float').values).float()
            self.tags[question_id] = torch.from_numpy(row.tags.astype('float')).float()
        gc.collect()
        
        userIds = userInteractions.user_id.unique()[:numberOfTrainingUsers]
        dataWithUsers = cleanUpUserInteractions(userInteractions[userInteractions.answered_correctly != -1].loc[userInteractions.user_id.isin(userIds)], garbageCollect=True)
        
        self.blankTags = torch.from_numpy(convertTagsToEncodedArray([])).float()
        self.blankQuestion = torch.from_numpy(np.ones(len(questionFeatures),)).float()
        
        gc.collect()
        
        self.length = len(userIds)
        dataBank = []

        for userId in userIds:
            data = []
            for index,row in dataWithUsers.loc[dataWithUsers.user_id == userId].iterrows():
                data.append([*self.getData(row), torch.Tensor([row.answered_correctly]).long()])
            dataBank.append(data)
        
        self.dataBank = dataBank
        
        gc.collect()
        

    
    def getData(self, row):
        questionId = int(row.content_id)
        return torch.from_numpy(row[interactionFeatures].astype('float').values).float(), self.questions.get(questionId, self.blankQuestion), self.tags.get(questionId, self.blankTags)
        
    def __len__(self): 
        return self.length
    
    def __getitem__(self, idx):
        return self.dataBank[idx]

gc.collect()

0

In [10]:
data = UserDataset(userActions, questions)
gc.collect()

0

In [11]:
data[0]

[[tensor([ 0.,  0.,  1., -1., -1.]),
  tensor([5.6920e+03, 5.0000e+00]),
  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]),
  tensor([1])],
 [tensor([5.6943e+04, 0.0000e+00, 2.0000e+00, 3

In [12]:
batch_size = 1

gc.collect()

train_dataset, val_dataset = random_split(data, [900, 100])
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                        shuffle=False)

In [13]:
hiddenSateDimension = 10

class Predictor(nn.Module):
  def __init__(self, tagCount: int, questionFeatureCount: int, interactionFeatureCount: int):
    super(Predictor, self).__init__()
    
    reductionDimensions = 4
    predictorDimensions = 16
    

    self.tagEncoder = nn.Sequential(
      nn.Linear(tagCount, 64),
      nn.ReLU(True),
      nn.Linear(64, 32),
      nn.ReLU(True), 
      nn.Linear(32, reductionDimensions)
    )

    self.tagDecoder = nn.Sequential(
      nn.Linear(reductionDimensions, 32),
      nn.ReLU(True),
      nn.Linear(32, 64),
      nn.ReLU(True),
      nn.Linear(64, tagCount), 
#       nn.Tanh()
    )

    self.predictorEncoder = nn.Sequential(
      nn.Linear(reductionDimensions + questionFeatureCount + interactionFeatureCount + hiddenSateDimension, 128),
      nn.ReLU(True),
      nn.Linear(128, 64),
      nn.ReLU(True),
      nn.Linear(64, 32),
      nn.ReLU(True),
      nn.Linear(32, predictorDimensions),
    )
    
    self.hiddenStateCreator = nn.Sequential(
      nn.Linear(predictorDimensions, 32),
      nn.ReLU(True),
      nn.Linear(32, 64),
      nn.ReLU(True),
      nn.Linear(64, 128),
      nn.ReLU(True),
      nn.Linear(128, hiddenSateDimension),  
    )
    
    self.predictor = nn.Sequential(
        nn.Linear(predictorDimensions, 32),
        nn.ReLU(True),
        nn.Linear(32, 16),
        nn.ReLU(True),
        nn.Linear(16, 2),
        nn.Tanh()
#         nn.Softmax()
    )
    
  def getBlankHiddenState(self):
    return Variable(torch.zeros(1, hiddenSateDimension))
    


  def forward(self, interaction, question, tags, hiddenState):
    if hiddenState is None:
        hiddenState = Variable(self.getBlankHiddenState())
    encodedTag = self.tagEncoder(tags)
    decodedTag = self.tagDecoder(encodedTag)
    predictOn = torch.cat((encodedTag, question, interaction, hiddenState), -1)
    predictEncoded = self.predictorEncoder(predictOn)
    prediction = self.predictor(predictEncoded)
    newHiddenState = self.hiddenStateCreator(predictEncoded)
    return prediction, decodedTag, newHiddenState


In [14]:
lossBase = nn.MSELoss()
classificationLoss = nn.CrossEntropyLoss()

def loss_criterion(tags, decodedTags, tagRegularizationTerm, answeredCorrectly, prediction):
  return lossBase(tags, decodedTags) * tagRegularizationTerm + classificationLoss(prediction, answeredCorrectly.squeeze(1))

In [15]:
tagRegularizationTerm = 0.001
predictionRegularizationTerm = 0.001
epochs = 2

sae = Predictor(maximumTag + 1, len(questionFeatures), len(interactionFeatures))
sae_optimizer = torch.optim.Adam(
    sae.parameters(), lr=0.001, weight_decay=1e-5)


train_loss  = []
train_acc = []
val_loss = []
val_acc = []

gc.collect()

for epoch in range(epochs):
    sae.train()
    running_acc = 0.0
    batch_loss = []
    for index, samples in enumerate(train_loader):
        if index % 100 == 0:
            gc.collect()
        hiddenState = None
        loss = None
        for interaction, question, tags, y in samples:
            interaction = Variable(interaction)
            question = Variable(question)
            tags = Variable(tags)
            y = Variable(y)
            # ===================forward=====================
            (prediction, decodedTag, newHiddenState) = sae(interaction, question, tags, hiddenState)
            currentLoss = loss_criterion(tags, decodedTag, tagRegularizationTerm, y, prediction)
            if loss is None:
                loss = currentLoss
            else:
                loss += currentLoss
            hiddenState = newHiddenState
        # ===================backward====================
        sae_optimizer.zero_grad()
        if loss is not None:
            loss.backward()
        sae_optimizer.step()

        # print statistics
        batch_loss.append(loss.item())
        
        out = torch.argmax(prediction.detach(),dim=1).unsqueeze(1)
        assert out.shape==y.shape
        running_acc += (out==y).sum().item()
    train_loss.append(np.mean(batch_loss))
    train_acc.append(running_acc*100/len(train_dataset))
    print(f"Train loss {epoch+1}: {train_loss[-1]},Train Acc:{running_acc*100/len(train_dataset)}%")



    sae.eval()
    batch_loss  = []
    correct = 0.0
    sampleCount = 0
    with torch.no_grad():
        totalLoss = 0
        for index, samples in enumerate(val_loader):
            hiddenState = None
            sampleCount += len(samples)
            for interaction, question, tags, y in samples:
                interaction = Variable(interaction)
                question = Variable(question)
                tags = Variable(tags)
                y = Variable(y)

                (prediction, decodedTag, newHiddenState) = sae(interaction, question, tags, hiddenState)
                loss = loss_criterion(tags, decodedTag, tagRegularizationTerm, y, prediction).item()
                
                hiddenState = newHiddenState

                batch_loss.append(loss)

                out = torch.argmax(prediction,dim=1).unsqueeze(1)
                acc = (y==out).sum().item()
                correct += acc
    val_loss.append(np.mean(batch_loss))
    val_acc.append(correct*100/sampleCount)
    print(f"Val accuracy:{correct*100/sampleCount}% Val loss:{np.mean(batch_loss)}")
    
    gc.collect()



Train loss 1: 217.81039434214432,Train Acc:51.111111111111114%
Val accuracy:68.50656660412758% Val loss:0.7568070019728471
Train loss 2: 229.7186310434507,Train Acc:51.111111111111114%
Val accuracy:68.50656660412758% Val loss:0.7568064323157128


In [16]:
print(train_loss)
print(train_acc)
print(val_loss)
print(val_acc)

[217.81039434214432, 229.7186310434507]
[51.111111111111114, 51.111111111111114]
[0.7568070019728471, 0.7568064323157128]
[68.50656660412758, 68.50656660412758]


In [17]:
env = riiideducation.make_env()

In [18]:
hiddenStates = {}
sae.eval()

with torch.no_grad():
    for (test_df, sample_prediction_df) in env.iter_test():
        test_df = cleanUpUserInteractions(test_df, removeNoneQuestions=True, garbageCollect=True)
        n = len(test_df)
        answeredCorrectly = [0] * n
        index = 0
        for _, row in test_df.iterrows():
            hiddenState = hiddenStates.get(row.user_id, None)
            interaction, question, tags = data.getData(row)
            interaction = Variable(interaction.unsqueeze(0))
            question = Variable(question.unsqueeze(0))
            tags = Variable(tags.unsqueeze(0))
            prediction, decodedTag, newHiddenState = sae(interaction, question, tags, hiddenState)
            hiddenStates[row.user_id] = newHiddenState
            prediction = torch.abs(prediction.squeeze(0))
            prediction = float(prediction[1].item() / prediction.sum().item())
            answeredCorrectly[index] = prediction
            index += 1
        test_df['answered_correctly'] = answeredCorrectly
        env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])